In [1]:
import os
from pathlib import Path
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from pylibs.plot_utils import set_size
from pylibs.setuparea import setarea
from pylibs.utils import get_dates

In [2]:
tlsize = 12
txsize = 12
mpl.rc('axes', titlesize=tlsize, labelsize=txsize)
mpl.rc('xtick', labelsize=txsize)
mpl.rc('ytick', labelsize=txsize)
mpl.rc('legend', fontsize='small')

In [3]:
axe_w = 4; axe_h = 4; plot_quality = 600
# Level control
vmin = 0; vmax = 1.e-4
# Colorbar control
cb_ori = 'vertical'
cb_frac = 0.025
cb_pad = 0.04
cb_asp = 32
# Area control
area = 'wxaq'

sub_areas = {
    'Boston': [-72., -70., 42., 43.],
    'NewYork': [-75.6, -72.4, 39.8, 41.4],
    'Toronto': [-80., -78., 43., 44.],
}
minlon = -81.; maxlon = -70; minlat = 39.8; maxlat = 46.

srcpath = '/glade/work/swei/Git/JEDI-METplus'

plot_product = 'wrfchem_evaluate'
obs_name = 'pandora_no2_total-wxaq'

# JEDI variable name, var:channel 
plot_var = 'nitrogendioxideTotal'

vrfy_fhr = 19
sdate = 2024082419
edate = 2024082419
hint = 24
dates = get_dates(sdate, edate, hint)

unit_str = 'mol m$^{-2}$'

hofx_path = os.path.join(srcpath, 'output', plot_product, 'hofx',
                         'f%.2i' % (vrfy_fhr), obs_name)
plts_path = os.path.join(srcpath, 'output', plot_product, 'plots', '2dmap', obs_name)

if not os.path.exists(hofx_path):
    raise Exception(f'HofX folder: {hofx_path} is not available')

if not os.path.exists(plts_path):
    os.makedirs(plts_path)

# Setup projection
if area == 'wxaq':
    proj = ccrs.LambertConformal(central_longitude=-97.0,
                                 central_latitude=39.0,
                                 standard_parallels=[30., 60.])
if area == 'glb':
    proj = ccrs.PlateCarree()

In [4]:
for cdate in dates:
    plot_date = cdate.strftime('%Y%m%d%H')
    hofx_file = f'hofx.{obs_name}.{plot_date}.nc4'
    print(f'Processing: {hofx_file}')
    in_hofx = os.path.join(hofx_path, hofx_file)
    if not os.path.exists(in_hofx):
        print(f'WARNING: Skip {plot_date}, {hofx_file} is not available')
        continue

    raw_ds = xr.open_dataset(in_hofx)
    meta_ds = xr.open_dataset(in_hofx,group='MetaData')
    lons = meta_ds.longitude
    lats = meta_ds.latitude
    
    obsval_ds = xr.open_dataset(in_hofx, group='ObsValue')
    hofx_ds = xr.open_dataset(in_hofx, group='hofx')
    preqc_ds = xr.open_dataset(in_hofx, group='PreQC')

    varname = plot_var
    df = meta_ds.to_dataframe()
    df['ObsValue'] = obsval_ds[varname].values
    df['hofx'] = hofx_ds[varname].values
    df['preqc'] = preqc_ds[varname].values
    pltdf = df.groupby(['latitude', 'longitude'])[['ObsValue', 'hofx']].mean().reset_index()
    
    for plot_type in ['ObsValue', 'H(X)']:
        if plot_type == 'ObsValue':
            pltcolumn = 'ObsValue'
        if plot_type == 'H(X)':
            pltcolumn = 'hofx'

        for sub_area_name in sub_areas.keys():
            minlon, maxlon, minlat, maxlat = sub_areas[sub_area_name]

            fig = plt.figure()
            ax = plt.subplot(projection=proj)
            set_size(axe_w, axe_h, t=0.96, l=0.14, b=0.1, r=0.86)
            ax.set_extent((minlon, maxlon, minlat, maxlat), crs=ccrs.PlateCarree())
            gl = ax.gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False)
            gl.xlines = False
            gl.ylines = False
            gl.xlabel_style = {'size': txsize - 1}
            gl.ylabel_style = {'size': txsize - 1}
            gl.right_labels = False
            gl.top_labels = False
            # gl.xformatter = LongitudeFormatter(degree_symbol=u'\u00B0 ')
            # gl.yformatter = LatitudeFormatter(degree_symbol=u'\u00B0 ')
            sc = ax.scatter(pltdf['longitude'], pltdf['latitude'], c=pltdf[pltcolumn], s=100,
                            vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree(),
                            cmap='managua_r', alpha=0.8, zorder=5)

            ax.add_feature(cft.BORDERS.with_scale('10m'), linestyle='--')
            if area == 'glb':
                ax.coastlines(resolution='10m')
            if area == 'wxaq':
                ax.add_feature(cft.STATES.with_scale('10m'))
                ax.add_feature(cft.LAKES.with_scale('10m'), facecolor='None',
                               edgecolor='k')

            title_str = f'{plot_type} at {plot_date} {sub_area_name}'
            cb_str = f'{plot_var} ({unit_str})'
            ax.set_title(title_str)
            cb = plt.colorbar(sc, orientation=cb_ori, fraction=cb_frac, pad=cb_pad, aspect=cb_asp, label=cb_str)
            cb.set_label(cb_str, fontsize=txsize)
            cb.ax.ticklabel_format(axis='y', style='sci', scilimits=(0, 0), useMathText=True)

            plotname = f'{sub_area_name}.{pltcolumn}.{varname}.f{vrfy_fhr}.{plot_date}.png'
            outname = os.path.join(plts_path, plotname)
            fig.savefig(outname, dpi=plot_quality)
            plt.close()

print('Process Finished')

Processing: hofx.pandora_no2_total-wxaq.2024082419.nc4
Process Finished


In [52]:
for sub_area_name in sub_areas.keys():
    minlon, maxlon, minlat, maxlat = sub_areas[sub_area_name]
    print(minlon, maxlon, minlat, maxlat)

-72.0 -70.0 42.0 43.0
-75.6 -72.4 39.8 41.4
-80.0 -78.0 43.0 44.0


In [25]:
sub_areas

{'Boston': [-72.0, -70.0, 42.0, 44.0],
 'NewYork': [-76.0, -73.0, 38.0, 41.0],
 'Toronto': [-80.0, -78.0, 42.5, 44.5]}